In [1]:
import fiftyone as fo
import fiftyone.brain as fob
import numpy as np
from glob import glob
import json
import os

In [2]:
dataset = fo.Dataset.from_images_dir('data', name=None, tags=None, recursive=True)

 100% |███████████| 106589/106589 [27.2s elapsed, 0s remaining, 5.3K samples/s]      


In [3]:
session = fo.launch_app(dataset, auto=False)
# # http://localhost:5151


Could not connect session, trying again in 10 seconds

Session launched. Run `session.show()` to open the App in a cell output.


In [4]:
session

Dataset:          2024.09.06.08.32.00
Media type:       image
Num samples:      106589
Selected samples: 0
Selected labels:  0
Session URL:      http://localhost:5151/

In [5]:
import os
import csv
def format_frame_id(frame_id):
    return str(frame_id).zfill(3)

metadata_dir = 'media-info'
# Đọc các file CSV và tạo dictionary ánh xạ
frame_idx_map = {}
csv_folder = 'map-keyframes'  # Thay đổi đường dẫn này nếu cần

for csv_file in os.listdir(csv_folder):
    if csv_file.endswith('.csv'):
        video_name = os.path.splitext(csv_file)[0]
        frame_idx_map[video_name] = {}
        
        with open(os.path.join(csv_folder, csv_file), 'r') as f:
            csv_reader = csv.reader(f)
            next(csv_reader)  # Bỏ qua header
            for row in csv_reader:
                frame_id = format_frame_id(int(row[0]))
                frame_idx = int(row[3])
                frame_idx_map[video_name][frame_id] = frame_idx

In [6]:
frame_idx_map

{'L01_V001': {'001': 0,
  '002': 125,
  '003': 300,
  '004': 434,
  '005': 467,
  '006': 597,
  '007': 717,
  '008': 877,
  '009': 893,
  '010': 1053,
  '011': 1218,
  '012': 1384,
  '013': 1558,
  '014': 1714,
  '015': 1832,
  '016': 2007,
  '017': 2109,
  '018': 2255,
  '019': 2428,
  '020': 2517,
  '021': 2668,
  '022': 2840,
  '023': 2927,
  '024': 3003,
  '025': 3117,
  '026': 3292,
  '027': 3417,
  '028': 3567,
  '029': 3742,
  '030': 3867,
  '031': 3960,
  '032': 4120,
  '033': 4212,
  '034': 4235,
  '035': 4362,
  '036': 4526,
  '037': 4682,
  '038': 4857,
  '039': 4955,
  '040': 5057,
  '041': 5183,
  '042': 5308,
  '043': 5433,
  '044': 5558,
  '045': 5658,
  '046': 5749,
  '047': 5924,
  '048': 5999,
  '049': 6174,
  '050': 6344,
  '051': 6430,
  '052': 6520,
  '053': 6656,
  '054': 6686,
  '055': 6844,
  '056': 6898,
  '057': 7000,
  '058': 7075,
  '059': 7125,
  '060': 7154,
  '061': 7222,
  '062': 7261,
  '063': 7362,
  '064': 7531,
  '065': 7612,
  '066': 7681,
  '067': 

In [7]:
for sample in dataset:
    _, sample['video'], sample['frameid'] = sample['filepath'][:-4].rsplit('\\', 2)
    id = sample['video']
    sample['frame_idx'] = (frame_idx_map[id])[sample['frameid']]
    metadata_file = os.path.join(metadata_dir, id + ".json")
    if os.path.exists(metadata_file):
        # Load the metadata from the JSON file
        with open(metadata_file, "r") as f:
            metadata = json.load(f)
        
    # Create a VideoMetadata object if you're dealing with video frames
    publish_date = metadata.get("publish_date", None)
    # Alternatively, for images, use fo.ImageMetadata(**metadata)
    if publish_date:
        sample["date"] = publish_date


    # Save the sample to update it in the dataset
    sample.save()

In [ ]:
all_keyframe = glob('data\\*\\*.jpg')
video_keyframe_dict = {}
all_video = glob('data\\*')
all_video = [v.rsplit('\\',1)[-1] for v in all_video]


In [ ]:
for kf in all_keyframe:
    _, vid, kf = kf[:-4].rsplit('\\',2)
    if vid not in video_keyframe_dict.keys():
        video_keyframe_dict[vid] = [kf]
    else:
        video_keyframe_dict[vid].append(kf)

In [ ]:
for k,v in video_keyframe_dict.items():
    video_keyframe_dict[k] = sorted(v)

In [ ]:
embedding_dict = {}
for v in all_video:
    clip_path = f'clip-features-32\\{v}.npy'
    a = np.load(clip_path)
    embedding_dict[v] = {}
    for i,k in enumerate(video_keyframe_dict[v]):
        embedding_dict[v][k] = a[i]


In [ ]:
clip_embeddings = []
for sample in dataset:
    clip_embedding = embedding_dict[sample['video']][sample['frameid']]
    clip_embeddings.append(clip_embedding)


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('clip-ViT-B-32')

c:\Users\Dell\anaconda3\envs\myenv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\Dell\anaconda3\envs\myenv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
fob.compute_similarity(
    dataset,
    model=model,      # store model's name for future use
    embeddings=clip_embeddings,          # precomputed image embeddings
    brain_key="img_sim",
)

In [ ]:

milvus_index = fob.compute_similarity(
    dataset,
    model="clip-vit-base32-torch",   
    embeddings=clip_embeddings,    
    brain_key="milvus_index",
    backend="milvus",
)

ValueError: Brain method run with key 'milvus_index' already exists

In [ ]:
query = dataset.first().id
view = dataset.sort_by_similarity(
    query,
    brain_key="milvus_index",
    k=100, 
)
